In [1]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.models import load_model
from random import randint, choice
import heapq

import sys
sys.path.insert(1, '../utils')
from NLP_little_helpers import *
little_helpers()

Using TensorFlow backend.
[nltk_data] Downloading package brown to /home/felipe/nltk_data...
[nltk_data]   Package brown is already up-to-date!


potions: re, numpy as np, pandas as pd, pickle, json, nltk, keras, collections
spells: clean_data, clean_entire, predict_tags, tagged_3D, tagged_n_grams, unknown_words_X, check_and_predict


In [2]:
with open('../Trained_Weights/hmm_model.pkl', 'rb') as file:
    hmm_model = pickle.load(file)
    
with open('../Trained_Weights/tokenizer_text.pkl', 'rb') as file:
    t_text = pickle.load(file)
    
with open('../Trained_Weights/tokenizer_tags.pkl', 'rb') as file:
    t_tags = pickle.load(file)
    
seq2word = dict(map(reversed, t_text.word_index.items()))
    
model = load_model('../Trained_Weights/tagged_quotes.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
def sample(preds, top_n=500):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [5]:
n = 5
max_length = 100
out_sentence = []
out_word = ''
original = []

while len(out_sentence) < max_length:
    #print(out_sentence)
    if len(out_sentence) == 0:
        
        text = ['<bos>']
        tags = ['<bos>']
        
    else:
        text, tags = predict_tags(out_sentence[-1], hmm_model)
        
    enc_texts = pad_sequences(t_text.texts_to_sequences([text]), maxlen=n)
    enc_tags = pad_sequences(t_tags.texts_to_sequences([tags]), maxlen=n)                             

    # compute probabilities for each word
    pred = model.predict([enc_texts, enc_tags])
    y_hat = np.argmax(pred)
    
    if len(out_sentence) == 0:
        y_hat = choice(sorted(enumerate(pred[0]),
                       key=lambda x: x[1]
                       )[-200:])[0]
        
        
    while seq2word[y_hat] in out_sentence[-4:] or y_hat in [0,4]:
        #y_hat = choice(sorted(enumerate(pred[0]),
         #              key=lambda x: x[1]
          #             )[-5:-1])[0]
        
        y_hat = choice(sample(pred[0])[:-490])
        #original.append(seq2word[y_hat])
        
    out_word = seq2word[y_hat]
    if out_word == '<eos>' and len(out_sentence) > 20:
        break
    else:    
        out_sentence += [out_word]
        
    
print(' '.join(out_sentence).replace('<eos>', ''))

two  with is on of  for is with of  and for with in  and of to is


In [ ]:
seq2word[5]

In [ ]:
original